In [1]:
import os
import json
from scrapper2 import get_contents, get_website_links
from IPython.display import Markdown, display

from dotenv import load_dotenv
from openai import OpenAI


In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Step1: narrowing down to only releavant links

### building the prompts

In [3]:
link_system_prompt = """
You are provided with ta list of links found on a webpage.
You are able to decide which of the links would be most relevant to include ina a brochure about eh company, 
such as links to an About page, or a Company page, or  Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url":"https://full.url/goes/here/about"},
        {"type": "careers page", "url":"https://full.url/careers"},
    ]
}
"""


def get_links_user_prompt(url):
    user_prompt = f"""
    Here is the list of links on the website {url} -
    Please decide which of these are relevant web links for a brochure about the company,
    respond with the full https URL in JSON format.
    Do not include Terms of SErvice, Privacy, email links.

    Links (some might be relative links):
    """

    links = get_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

### making the call to openai

In [4]:
def select_relevant_links(url):
    openai = OpenAI()
    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
       response_format={"type":"json_object"}
    )

    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links


#select_relevant_links("https://edwarddonner.com")

## Step2: making the brochure

In [5]:
def fetch_page_and_all_relevant_links(url):
    """
    this function iterates over the relevant links and get its contents
    """
    contents = get_contents("https://edwarddonner.com")
    relevant_links = select_relevant_links(url)
    result = f"## Landing page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += get_contents(link["url"])
    return result

In [6]:
#print(fetch_page_and_all_relevant_links("https://huggingface.co"))

## Step3: creating the prompts

In [7]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of sevral relevant pages from a company website
and creates a short brochure about the company for prospective customers, investor and recuits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you the information
"""

## or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate the 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of sevral relevant pages from a company website
# and creates a short, humorous , entertaining, witty brochure about the company for prospective customers, investor and recuits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you the information
# """


In [8]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
    You are looking at a company called: {company_name}
    Here are the contents of its landing page and other relevant pages;
    use this information to build a short brochure of the company in markdown without code blocks.\n\n
    """

    user_prompt  += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5000 characters
    return user_prompt

In [9]:
## checking how the user_prompt is looking like
#get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

## Step4: creating the brochure

In [10]:
def create_brochure(company_name, url):
    openai = OpenAI()
    response = openai.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = [
            {"role":"system", "content": brochure_system_prompt},
             {"role":"user", "content": get_brochure_user_prompt(company_name, url)}
        ]
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [11]:
create_brochure("HuggingFace", "https://huggingface.co")

Found 3 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is the leading collaboration platform for the global machine learning (ML) community. It serves as a central hub where ML engineers, researchers, scientists, and enthusiasts can **share, explore, discover, and experiment** with open-source machine learning models, datasets, and applications. Hugging Face empowers the next generation of AI developers and users to build an **open and ethical AI future**, driving innovation at the very heart of the AI revolution.

---

## What We Offer

- **Hugging Face Hub:**  
  A central place hosting over **1 million models, 250,000+ datasets, and 400,000+ applications** across multiple modalities such as text, image, video, audio, and 3D.

- **Open-source ML Stack:**  
  Tools and libraries that enable faster ML development — accelerating research and production deployment.

- **Spaces:**  
  A platform to create and share ML web apps, enabling community collaboration and experimentation in real-time.

- **Enterprise and Compute Solutions:**  
  Paid offerings providing scalable compute power and enterprise-grade solutions tailored for business teams and organizations.

---

## Company Culture

Hugging Face is powered by a passionate and diverse community dedicated to **collaborative learning and openness**. Transparency, inclusivity, and ethical AI development are core to their values. The company fosters a welcoming environment where people can build, share, and grow their AI expertise together.

They emphasize:

- **Community-driven Innovation:** Encouraging participation and sharing from beginners to experts across the globe.
- **Building an Ethical AI Future:** Committed to responsible and transparent AI research and deployment.
- **Empowerment:** Supporting the next generation of machine learning engineers and scientists.

---

## Our Customers and Community

Hugging Face has attracted a **fast-growing community** ranging from independent developers and researchers to large enterprises. Some highlights:

- **Researchers and Scientists** leverage ready-to-use models and datasets to accelerate their projects.
- **AI Engineers** use the platform to train, test, and deploy machine learning models.
- **Enterprises** benefit from customizable solutions and scale their AI infrastructure effectively.
- **Educators and Students** build portfolios and collaborate on open-source ML projects.
- **Creative Coders and Developers** explore multiple AI modalities with easy-to-use tools and APIs.

---

## Careers and Opportunities

Hugging Face is continuously expanding and welcomes talented individuals who are:

- Passionate about **machine learning and AI technologies**
- Committed to **open-source ethics and community collaboration**
- Interested in solving challenging problems at the **cutting edge of AI research**
- Eager to work in a **dynamic, inclusive, and mission-driven environment**

Positions cover a range of roles including research scientists, ML engineers, software developers, community managers, and enterprise solution architects.

Explore open roles and apply at: [huggingface.co/jobs](https://huggingface.co/jobs)

---

## Connect With Us

Stay up to date with the latest in AI and Hugging Face news:

- Website: [huggingface.co](https://huggingface.co)  
- GitHub: [github.com/huggingface](https://github.com/huggingface)  
- Twitter: [@huggingface](https://twitter.com/huggingface)  
- LinkedIn: [Hugging Face LinkedIn](https://linkedin.com/company/huggingface)  
- Discord Community: Join to discuss and collaborate with fellow ML enthusiasts

---

## Join the AI Revolution with Hugging Face

Whether you are a developer looking to build cutting-edge models, a researcher aiming to accelerate innovation, an organization scaling AI solutions, or a student building your AI career, Hugging Face offers the platform, tools, and community you need to succeed.

Together, let’s **build an open, ethical, and powerful AI future**.

---

**Hugging Face**  
*The AI community building the future.*